In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class bsm_options_value:

    def __init__(self, S0, K, T, r, sigma, option_type):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type

    def analytical(self):
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)

        if self.option_type == 'call':
            price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)

            delta = norm.cdf(d1)
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2))  # per business day
            rho = (self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2)) / 100

        elif self.option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)

            delta = norm.cdf(d1) - 1
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2))  # per business day
            rho = (-self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2)) / 100

        return price , delta, gamma, theta, vega, rho

    ############################################################################################################

    def plot_option_value_K(self, logK_array, option_price, option_price_at_K, refx, refy, ax):
        # Main option price curve
        ax.plot(np.exp(logK_array), option_price, label='Option Price vs Strike', color='blue')
        ax.plot(refx, refy, label='Option Price vs Strike', color='black')

        # Highlight the option price at strike K
        ax.scatter([(self.K)], [option_price_at_K], color='red', marker='o', label=f'Price at K={self.K:.2f}: {option_price_at_K:.4f}', zorder=5)
        ax.axvline(x=(self.K), color='green', linestyle='--', label=f'Strike Price $K^o$')
        ax.set_xlabel('Strike Price (K)')
        ax.set_ylabel('Option Value')
        ax.set_title('Option Price vs Strike (Carr–Madan)')
        ax.set_xlim(0.5*self.K, 1.5*self.K)
        ax.set_ylim(-0.0001, self.K*0.5)
        #ax.legend()
        ax.grid(True)

    def plot_option_value_logK(self, logK_array, option_price, option_price_at_K, ax):
        # Main option price curve
        ax.plot(logK_array, option_price, label='Option Price vs Strike', color='blue')

        # Highlight the option price at strike K
        ax.scatter([np.log(self.K)], [option_price_at_K], color='red', marker='o', label=f'Price at K={self.K:.2f}: {option_price_at_K:.4f}', zorder=5)
        ax.axvline(x=np.log(self.K), color='green', linestyle='--', label=f'Strike Price $logK^o$')
        ax.set_xlabel('Strike Price (logK)')
        ax.set_ylabel('Option Value')
        ax.set_title('Option Price vs log Strike (Carr–Madan)')
        ax.set_xlim(0.5*np.log(self.K), 1.5*np.log(self.K))
        ax.set_ylim(-0.0001, self.K*0.5)
        #ax.legend()
        ax.grid(True)

    def char_func_numerical(self, N):
        # Grid in log-space
        x_min = np.log(0.01 * self.S0)
        x_max = np.log(2 * self.S0)
        x = np.linspace(x_min, x_max, N)
        dx = x[1] - x[0]
        dx = x[1] - x[0]

        mu = np.log(self.S0) + (self.r - 0.5 * self.sigma**2) * self.T
        pdf = (1 / np.sqrt(2 * np.pi * self.sigma**2 * self.T)) * np.exp(- (x - mu)**2 / (2 * self.sigma**2 * self.T))
        phi_fft = np.fft.fft(pdf) * dx
        u = 2 * np.pi * np.fft.fftfreq(N, d=dx)

        u_shift, phi_fft_shift = np.fft.fftshift(u), np.fft.fftshift(phi_fft)
        cf_exact = self.char_func(u)

        plt.figure(figsize=(10, 6))
        plt.plot(u, np.real(cf_exact), label='Re[Analytical CF]', linestyle='--')
        plt.plot(u_shift, np.real(phi_fft_shift), label='Re[Numerical CF via FFT]')
        plt.xlim(-50, 50)
        plt.title('Comparison of Analytical and Numerical CF')
        plt.xlabel('u')
        plt.ylabel('Real Part')
        plt.legend()
        plt.grid()
        plt.show()


    def char_func(self, u):
        """Characteristic function φ(u) of log(S_T) under risk-neutral BSM model"""
        return np.exp(1j * u * (np.log(self.S0) + (self.r - 0.5 * self.sigma**2) * self.T) - 0.5 * self.sigma**2 * u**2 * self.T)

    def fft_cm(self, alpha=1.5, N=4096, eta=0.25):

        #self.char_func_numerical(N)

        # Frequency domain
        u = np.arange(N) * eta
        lambd = 2 * np.pi / (N * eta) #Nyquist

        # Compute log-strike grid
        k = np.arange(N) * lambd
        K_array = np.exp(k)

        # Dampened characteristic function
        phi = self.char_func(u - 1j * (alpha + 1))
        numerator = np.exp(-1j * u * k) * np.exp(-self.r * self.T) * phi
        denominator = alpha**2 + alpha - u**2 + 1j * (2 * alpha + 1) * u
        integrand = numerator / denominator
        integrand *=  eta / np.pi

        # Apply FFT
        fft_output = np.fft.fft(integrand).real

        # Get option values
        call_prices = fft_output * np.exp(-alpha * k)

        if self.option_type == 'call':
            option_price = call_prices
            interp_fn = interp1d(K_array, option_price, kind='cubic', bounds_error=False, fill_value='extrapolate')
            option_price_at_K = interp_fn(self.K)
        elif self.option_type == 'put':
            option_price = call_prices - self.S0 + K_array * np.exp(-self.r * self.T)
            interp_fn = interp1d(K_array, option_price, kind='cubic', bounds_error=False, fill_value='extrapolate')
            option_price_at_K = interp_fn(self.K)

        return option_price_at_K, k, option_price, np.log(self.K), option_price_at_K


    def chi_psi_vectorized(self, k, a, b, c, d):
        omega = k * np.pi / (b - a)      # omega_k
        delta = b - a                    # Precompute width
        # Arguments
        omega_c = omega * (c - a)
        omega_d = omega * (d - a)
        # Avoid division by zero for k=0 in psi
        omega_nonzero = omega.copy()
        omega_nonzero[0] = 1.0
        # ---- chi_k ----
        exp_c = np.exp(c)
        exp_d = np.exp(d)

        chi_k = (exp_d * (np.cos(omega_d) + omega * np.sin(omega_d)) - exp_c * (np.cos(omega_c) + omega * np.sin(omega_c)) )
        chi_k /= (1.0 + omega**2)

        # ---- psi_k ----
        psi_k = (np.sin(omega_d) - np.sin(omega_c)) / omega_nonzero

        # Correct psi_k[0] separately (limit as omega → 0)
        psi_k[0] = d - c

        return chi_k, psi_k

    def fft_cos(self, N=256, L=10):
        # Step 1: log-strike
        x0 = np.log(self.S0)

        # Step 2: Truncation range [a, b]
        c1 = x0 + (self.r - 0.5 * self.sigma**2) * self.T
        c2 = self.sigma**2 * self.T
        c4 = 3 * self.sigma**4 * self.T**2

        a = c1 - L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))
        b = c1 + L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))

        k = np.arange(N)
        u = k * np.pi / (b - a)

        if self.option_type == 'call':
            c, d = np.log(self.K), b
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (chi - self.K * psi)
        elif self.option_type == 'put':
            c, d = a, np.log(self.K)
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (self.K * psi - chi)
        else:
            raise ValueError("Option type must be 'call' or 'put'.")

        Vk[0] *= 0.5  # k=0 term is halved

        # Step 4: COS formula
        phi = self.char_func(u)
        payoff = np.real( (2  / (b-a)) * phi * np.exp(-1j * u * a)) * Vk
        price = np.exp(-self.r * self.T) * np.sum(payoff)

        return price

    ############################################################################################################
    def plot_option_value(self, S, V, S_T, option_price_at_ST, ax):
        ax.plot(S, np.maximum(S - self.K, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, np.maximum(self.K - S, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, V, color = 'blue', label='Option Value vs Stock Price at present')
        ax.axvline(x=self.K, color='red', linestyle='--', label='Strike Price K')
        ax.axvline(x=S_T, color='green', linestyle='--', label=f'Stock Price $S_0$')
        ax.scatter([S_T], [option_price_at_ST], color='black', zorder=5, label=f'Option Value at $S_0$')

        ax.set_title('Option Value vs Stock Price at Maturity')
        ax.set_xlabel('Stock Price S')
        ax.set_ylabel('Option Value V')
        ax.set_xlim(self.K - 10 , self.K + 10)
        ax.set_ylim(0, self.K)
        #ax.legend()
        ax.grid(True)

    def explicit_fd(self, exercise_type, exercise_days):
        M = 100
        N = 10000
        S_max = 2 * self.S0
        dS = S_max / M
        dt = self.T / N

        # Stability check
        stability_limit = (dS ** 2) / (self.sigma ** 2 * S_max ** 2)
        if dt > stability_limit:
            print("WARNING: The scheme may be unstable! Decrease dt or increase M.")

        # Stock price grid
        S = np.linspace(0, S_max, M + 1)

        # Terminal payoff
        if self.option_type == 'call':
            V = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            V = np.maximum(self.K - S, 0)
        else:
            raise ValueError("option_type must be 'call' or 'put'")

        payoff = V.copy()

        # Coefficients for explicit scheme
        alpha = 0.5 * self.sigma ** 2 * (S / dS) ** 2
        beta = 0.5 * self.r * (S / dS)

        # Convert Bermudan exercise days to time step indices
        if exercise_type == 'bermudan':
            exercise_indices = set([
                int((self.T - t_ex) / dt)
                for t_ex in exercise_days
            ])


        # Backward time-stepping
        for n in range(N):
            V_new = V.copy()

            for i in range(1, M):
                A = dt * (alpha[i] - beta[i])
                B = 1 - dt * (self.sigma**2 * (S[i] / dS)**2 + self.r)
                C = dt * (alpha[i] + beta[i])
                V_new[i] = A * V[i - 1] + B * V[i] + C * V[i + 1]

            # Boundary conditions
            t = self.T - n * dt
            if self.option_type == 'call':
                V_new[0] = 0
                V_new[-1] = S_max - self.K * np.exp(-self.r * t)
            else:
                V_new[0] = self.K * np.exp(-self.r * t)
                V_new[-1] = 0

            # Early exercise projection
            if exercise_type == 'american':
                V = np.maximum(V_new, payoff)
            elif exercise_type == 'bermudan' and t_index in exercise_days:
                V = np.maximum(V_new, payoff)
            else:
                V = V_new

        # Interpolate option price at S0
        option_price_at_ST = np.interp(self.S0, S, V)
        return option_price_at_ST, S, V, self.S0, option_price_at_ST

    def implicit_fd(self, exercise_type, exercise_days):
        max_iter = 100
        tol = 1e-6
        omega = 1.9

        M = 100
        N = 10000
        S_max = 2 * self.S0
        dS = S_max / M
        dt = self.T / N
        S = np.linspace(0, S_max, M + 1)

        if exercise_type == 'european':
            exercise_binary = np.zeros(N + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(N + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(N + 1)
            for i in range(N+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Terminal payoff
        if self.option_type == 'call':
            V = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            V = np.maximum(self.K - S, 0)

        # Prepare tridiagonal matrix coefficients
        i = np.arange(1, M)
        a = -0.5 * dt * (self.sigma**2 * i**2 - self.r * i)
        b = 1 + dt * (self.sigma**2 * i**2 + self.r)
        c = -0.5 * dt * (self.sigma**2 * i**2 + self.r * i)

        A_lower = a.copy()
        A_diag  = b.copy()
        A_upper = c.copy()

        for n in range(N):
            t = self.T - n * dt
            rhs = V[1:M]

            # Boundary conditions
            if self.option_type == 'call':
                V[0] = 0
                V[-1] = S_max - self.K * np.exp(-self.r * t)
            else:
                V[0] = self.K * np.exp(-self.r * t)
                V[-1] = 0

            rhs[0]  += a[0] * V[0]
            rhs[-1] += c[-1] * V[-1]

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                # PSOR for American option
                x = V[1:M].copy()
                phi = np.maximum(S[1:M] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S[1:M], 0)

                for k in range(max_iter):
                    x_old = x.copy()
                    for j in range(M - 1):
                        left  = A_lower[j] * (x[j - 1] if j > 0 else 0)
                        right = A_upper[j] * (x[j + 1] if j < M - 2 else 0)
                        x_new = (1 - omega) * x[j] + omega / A_diag[j] * (rhs[j] - left - right)
                        x[j] = max(x_new, phi[j])  # projection

                    if np.linalg.norm(x - x_old, np.inf) < tol:
                        break
                else:
                    print("Warning: PSOR did not converge")
            else:
                '''
                # Standard backward implicit solver for European option
                A = np.zeros((M - 1, M - 1))
                np.fill_diagonal(A, A_diag)
                np.fill_diagonal(A[1:], A_lower[1:])
                np.fill_diagonal(A[:, 1:], A_upper[:-1])
                x = np.linalg.solve(A, rhs)
                '''
                # Prepare banded matrix for scipy.linalg.solve_banded
                A = np.zeros((3, M-1))
                A[0, 1:]  = A_upper[:-1]      # Upper diagonal
                A[1, :]   = A_diag            # Main diagonal
                A[2, :-1] = A_lower[1:]       # Lower diagonal

                # Solve the linear system A x = rhs using banded solver
                x = solve_banded((1, 1), A, rhs)

            V[1:M] = x


        # Interpolation to get price at S0
        option_price_at_S0 = np.interp(self.S0, S, V)
        return option_price_at_S0, S, V, self.S0, option_price_at_S0


    def crank_nicolson_fd(self, exercise_type, exercise_days):
        max_iter = 100
        tol = 1e-6
        omega = 1.9

        M = 100
        N = 10000
        S_max = 2 * self.S0
        dS = S_max / M
        dt = self.T / N
        S = np.linspace(0, S_max, M + 1)

        if exercise_type == 'european':
            exercise_binary = np.zeros(N + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(N + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(N + 1)
            for i in range(N+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Terminal payoff
        if self.option_type == 'call':
            V = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            V = np.maximum(self.K - S, 0)

        # Prepare tridiagonal matrix coefficients
        i = np.arange(1, M)
        a = -0.5 * dt * (self.sigma**2 * i**2 - self.r * i)
        b = 1 + dt * (self.sigma**2 * i**2 + self.r)
        c = -0.5 * dt * (self.sigma**2 * i**2 + self.r * i)

        A_lower = a.copy()
        A_diag  = b.copy()
        A_upper = c.copy()

        # Matrix B (explicit part)
        i = np.arange(1, M)
        alpha = 0.5 * dt * (self.sigma**2 * i**2 - self.r * i)
        beta  = 1 - dt * (self.sigma**2 * i**2 + self.r)
        gamma = 0.5 * dt * (self.sigma**2 * i**2 + self.r * i)

        B_lower = alpha.copy()
        B_diag = beta.copy()
        B_upper = gamma.copy()

        for n in range(N):
            t = self.T - n * dt

            rhs = B_lower * V[:-2] + B_diag * V[1:-1] + B_upper * V[2:]

            # Boundary conditions
            if self.option_type == 'call':
                V[0] = 0
                V[-1] = S_max - self.K * np.exp(-self.r * t)
            else:
                V[0] = self.K * np.exp(-self.r * t)
                V[-1] = 0

            rhs[0]  += a[0] * V[0]
            rhs[-1] += c[-1] * V[-1]

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                # PSOR for American option
                x = V[1:M].copy()
                phi = np.maximum(S[1:M] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S[1:M], 0)

                for k in range(max_iter):
                    x_old = x.copy()
                    for j in range(M - 1):
                        left  = A_lower[j] * (x[j - 1] if j > 0 else 0)
                        right = A_upper[j] * (x[j + 1] if j < M - 2 else 0)
                        x_new = (1 - omega) * x[j] + omega / A_diag[j] * (rhs[j] - left - right)
                        x[j] = max(x_new, phi[j])  # projection

                    if np.linalg.norm(x - x_old, np.inf) < tol:
                        break
                else:
                    print("Warning: PSOR did not converge")
            else:
                '''
                # Standard backward implicit solver for European option
                A = np.zeros((M - 1, M - 1))
                np.fill_diagonal(A, A_diag)
                np.fill_diagonal(A[1:], A_lower[1:])
                np.fill_diagonal(A[:, 1:], A_upper[:-1])
                x = np.linalg.solve(A, rhs)
                '''
                # Prepare banded matrix for scipy.linalg.solve_banded
                A = np.zeros((3, M-1))
                A[0, 1:]  = A_upper[:-1]      # Upper diagonal
                A[1, :]   = A_diag            # Main diagonal
                A[2, :-1] = A_lower[1:]       # Lower diagonal

                # Solve the linear system A x = rhs using banded solver
                x = solve_banded((1, 1), A, rhs)

            V[1:M] = x

        # Interpolation to get price at S0
        option_price_at_S0 = np.interp(self.S0, S, V)
        return option_price_at_S0, S, V, self.S0, option_price_at_S0


    def compute_greeks_fd(self, S, V, method):
        dS = S[1] - S[0]

        # Find closest index to S0
        idx = np.searchsorted(S, self.S0)
        if idx < 1 or idx >= len(S) - 1:
            print("S0 is too close to the boundary for finite difference.")
            return

        # First Derivative: Delta
        delta = (V[idx + 1] - V[idx - 1]) / (2 * dS)

        # Second Derivative: Gamma
        gamma = (V[idx + 1] - 2 * V[idx] + V[idx - 1]) / (dS ** 2)

        # Theta: Approximate using terminal payoff and one time step back
        dt = self.T / 1000
        S_val = S[idx]
        if self.option_type == 'call':
            payoff = max(S_val - self.K, 0)
        else:
            payoff = max(self.K - S_val, 0)
        theta = -(V[idx] - payoff) / dt  # Finite diff in time direction

        # Vega: Bump sigma and reprice using the same method
        eps = 1e-4
        orig_sigma = self.sigma
        self.sigma += eps
        if method == 'explicit_fd':
            V_eps, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps, _, _, _, _  = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps, _, _, _, _  = self.crank_nicolson_fd()
        else:
            raise ValueError("Invalid method for computing Vega")
        self.sigma = orig_sigma
        vega = (V_eps - V[idx]) / eps

        # Rho: Bump interest rate and reprice
        orig_r = self.r
        self.r += eps
        if method == 'explicit_fd':
            V_eps_r, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps_r, _, _, _, _ = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps_r, _, _, _, _ = self.crank_nicolson_fd()

        self.r = orig_r
        rho = (V_eps_r - V[idx]) / eps

        return delta, gamma, theta, vega, rho